# Segmenting and Clustering Neighborhoods in Toronto

This notebook is divided in three sections, as required in the assignement.

## SECTION 1

### Getting data from Wikipedia

First of all we import required libraries

In [38]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Next we use requests and BeautifulSoap to get HTML from Wikipedia 

In [39]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

Uncommenting and running the following line gives a long text output: we use it to find the class name of the table we want to parse 

In [ ]:
#print(soup.prettify())

The above line of code let us discover the class name: wikitable.     
Now we use the name of the class to focus only on the table:

In [40]:
tb = soup.find('table', class_='wikitable')

Now tb has just the table HTML and content.    
We use pandas to turn table content into a pandas data frame:

In [41]:
df = pd.read_html(str(tb))[0]
df.shape

(180, 3)

Now we remove all the lines where Borough is 'Not assigned' and check how many lines remain:

In [42]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.shape

(103, 3)

Next, we change column names. Basically we remove the white space from 'Postal Code':

In [43]:
df.columns = [c.replace(' ', '') for c in df.columns]
df.columns.values

array(['PostalCode', 'Borough', 'Neighborhood'], dtype=object)

Now, we know that the data frame has 103 lines, so let's check that we have 103 unique postal codes:

In [44]:
len(df.PostalCode.unique())

103

Last check: see if there is any not assigned neighborhood:

In [45]:
len(df[df.Neighborhood=='Not assigned'])

0

**Here is the final data frame:**

In [46]:
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**Here is the shape of the data frame:**

In [47]:
df.shape

(103, 3)

## SECTION 2

### Getting Latitude and Longitude

We use the provided csv file, because other possible resources are unstable. The easiest way first!

In [33]:
url2='http://cocl.us/Geospatial_data'
ll=pd.read_csv(url2)
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now just a rapid check that this file has the same number of rows (103) that we have in the Toronto postal codes data frame:

In [29]:
len(ll)

103

In [34]:
ll.columns = [c.replace(' ', '') for c in ll.columns]
ll.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
new_df = pd.merge(df, ll, on='PostalCode')

**Here is the new data frame with Latitudes and Longitudes:**

In [52]:
new_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## SECTION 3

### Exploration and clustering of Toronto neighborhoods

First of all let's see how many boroughs and neighborhoods we have:

In [54]:
NB = new_df
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(NB['Borough'].unique()),
        NB.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


Let's import required libraries:

In [55]:
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans
import folium

Now we find the coordiantes of Toronto, in order to define the center point of our map:

address = 'Toronto'

geolocator = Nominatim(user_agent="trn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Now we are ready to plot our first map, showing Toronto neighborhoods. Each dot is a neighborhood and clicking on a we can see the names of neghborhood and borough:

In [73]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(NB['Latitude'], NB['Longitude'], NB['Borough'], NB['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now we focus on Downtown Toronto, which coordinates are the same used for Toronto itself:

In [61]:
dt_data = NB[NB['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dt_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [81]:
# create map of Downtown Toronto using latitude and longitude values
map_dt = folium.Map(location=[latitude, longitude], zoom_start=13.5)

# add markers to map
for lat, lng, label in zip(dt_data['Latitude'], dt_data['Longitude'], dt_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#FF001F',
        fill=True,
        fill_color='#FFAE00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt)  
    
map_dt

We list all the neighborhood we have in Downtown Toronto:

In [79]:
dt_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


Now let's focus even deeper. We choose neighborhood "Richmond, Adelaide, King":

In [83]:
dt_data.loc[7, 'Neighborhood']
neighborhood_latitude = dt_data.loc[7, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dt_data.loc[7, 'Longitude'] # neighborhood longitude value

neighborhood_name = dt_data.loc[7, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Richmond, Adelaide, King are 43.65057120000001, -79.3845675.


Now it's time to use Foursquare! We start defining credentials and parameters. We want to get the list of fitrst 100 venues in a radius of 500:

In [85]:
CLIENT_ID = '***' # hidden
CLIENT_SECRET = '***' # hidden
VERSION = '20200508' # Foursquare API version
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [ ]:
Well, now run the query on Foursquare servers:

In [87]:
results = requests.get(url).json()
#results # uncommenting this will print the long json with our results 

Let's define a function that will extract the category of each venue:

In [88]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Great! Now we can use the function to get a pandas dataframe with venues, categories and coordinates:

In [91]:
from pandas import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
3,The Keg Steakhouse + Bar - York Street,Restaurant,43.649987,-79.384103
4,Cafe Landwer,Café,43.648753,-79.385367


How many venues we have..?

In [92]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

94 venues were returned by Foursquare.


Now we extend the same process to all the neighborhoods in Downtown Toronto. We will define a function to loop the process:

In [93]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Great, now let's use tha function!

In [94]:
dt_venues = getNearbyVenues(names=dt_data['Neighborhood'],
                                   latitudes=dt_data['Latitude'],
                                   longitudes=dt_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


    
A quick check about the size and unique categories of our new dataframe:

In [96]:
print(dt_venues.shape)
print('There are {} uniques categories.'.format(len(dt_venues['Venue Category'].unique())))
dt_venues.head()

(1228, 7)
There are 210 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### Prepare data for some clustering

In [101]:
# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = dt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

Now let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [102]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()

Now we write a function to sort the venues in descending order:

In [103]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Brilliant! now let's use this function to create the new dataframe and display the top 10 venues for each neighborhood:**

In [105]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Bakery,Seafood Restaurant,Café,Cheese Shop,Pub,Breakfast Spot
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Plane
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Ice Cream Shop,Japanese Restaurant,Thai Restaurant,Burger Joint,Bubble Tea Shop
3,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Athletics & Sports,Candy Store,Restaurant,Italian Restaurant,Diner
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Smoke Shop,Gay Bar,Hotel,Gastropub,Mediterranean Restaurant


### We are ready for clustering!

We use K-means from sklearn. We want to create some clusters to group similar neighborhoods. Similar means with similar venues.

In [144]:
# set number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood:

In [145]:
# add clustering labels
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [146]:
dt_merged = dt_data

dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [147]:
dt_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Restaurant,Theater,Café,Bank,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Gym,Diner,Park,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Restaurant,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Theater
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Gym,Beer Bar,Restaurant,Seafood Restaurant,Italian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Bakery,Seafood Restaurant,Café,Cheese Shop,Pub,Breakfast Spot


### Let's see our clusters on a map!

In [148]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighborhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters